In [32]:
from htmd import *

In [57]:
tleap_in=b"""
source leaprc.ff14SB
m = sequence { ACE ALA NME }
saveAmberParm m diala.prmtop diala.rst7
savepdb m diala.pdb
quit
"""

from subprocess import Popen, PIPE, STDOUT
tleap_out,tleap_err = Popen(['tleap','-f','-'], stdout=PIPE, stdin=PIPE, stderr=PIPE)\
    .communicate(input=tleap_in)

if tleap_err != b'':
    print("****** Something went wrong, STDERR follows ******")
    print(tleap_err.decode())
    print("****** STDOUT follows ******")

print(tleap_out.decode())


-I: Adding /Users/toni/anaconda3/bin/../dat/leap/prep to search path.
-I: Adding /Users/toni/anaconda3/bin/../dat/leap/lib to search path.
-I: Adding /Users/toni/anaconda3/bin/../dat/leap/parm to search path.
-I: Adding /Users/toni/anaconda3/bin/../dat/leap/cmd to search path.
-f: Source -.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: /Users/toni/anaconda3/bin/../dat/leap/cmd/leaprc
----- Source: /Users/toni/anaconda3/bin/../dat/leap/cmd/leaprc.ff14SB
----- Source of /Users/toni/anaconda3/bin/../dat/leap/cmd/leaprc.ff14SB done
Log file: ./leap.log
Loading parameters: /Users/toni/anaconda3/bin/../dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /Users/toni/anaconda3/bin/../dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /Users/toni/anaconda3/bin/../dat/leap/lib/amino12.lib
Loading library: /Users/toni

In [58]:
%qtconsole

In [97]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

prmtop = AmberPrmtopFile('diala.prmtop')
inpcrd = AmberInpcrdFile('diala.rst7')

In [99]:
system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, 
                             nonbondedCutoff=1*nanometer,
                             constraints=HBonds)

In [101]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [102]:
platform = mm.Platform.getPlatformByName('CPU')
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

PDBReporter("pre.pdb",1).report(simulation,simulation.context.getState(getPositions=True))

In [103]:
simulation.minimizeEnergy()
PDBReporter("mini.pdb",1).report(simulation,simulation.context.getState(getPositions=True))



simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,130.9051091321744,189.6424658522868
2000,109.16815137863159,324.81944345351064
3000,134.35766665171832,306.08951220367265
4000,123.53904725424945,272.59757735578546
5000,131.94062200764165,311.15243490637977
6000,120.44675515592098,351.17569266132733
7000,112.02134200633736,287.13521303158785
8000,118.60111713083461,319.91233037299156
9000,134.9045149856247,355.3358195889588
10000,119.91769239906222,250.73477387320276
